# Step 1: Process CRC data
## Step 1.1: CRC data access
The CRC data was published by [Bian et al. ](https://www.science.org/doi/10.1126/science.aao3791) in 2018, and the raw data has been submitted to the NCBI GEO under accession number [GSE97693](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE97693)

## Step 1.2 scDNA-seq processing
As stated by [Bian et al. ](https://www.science.org/doi/10.1126/science.aao3791), CNV was estimated from single-cell DNA methylome sequencing data. We recommend using [bwa-meth](https://github.com/brentp/bwa-meth) for sequence alignment of BS-seq reads. Then [Samtools](http://www.htslib.org/) can be used for sorting and deduplicating. ANd the last step is using [featureCounts](https://subread.sourceforge.net/featureCounts.html#:~:text=featureCounts%20is%20a%20highly%20efficient,and%20genomic%20DNA%2Dseq%20reads.) to count the reads.


[Bian et al. ](https://www.science.org/doi/10.1126/science.aao3791) used [Bismark](https://www.bioinformatics.babraham.ac.uk/projects/bismark/#:~:text=Bismark%20is%20a%20program%20to,of%20their%20samples%20straight%20away.) for alignment, [Samtools](http://www.htslib.org/) for deduplicating, [bedtools](https://bedtools.readthedocs.io/en/latest/) for converting BAM files to BED files, and [Ginkgo algorithm](https://www.nature.com/articles/nmeth.3578) for constructing copy-number profiles. Details can be found in the supplementary materials of the original paper under the "SCNA estimation from single-cell DNA methylation sequencing data" section.

## Step 1.3 scRNA-seq processing
In the comparison experiments between MaCroDNA and other methods, scRNA-seq data from patients CRC01, CRC10, and CRC11 were applied. These data were performed according to the [multiplexed scRNA-seq method](https://doi.org/10.1186/s13059-018-1416-2) and the alignment was done by using [Tophat](http://ccb.jhu.edu/software/tophat/index.shtml). Details can be found in the supplementary materials of the original paper under the "Single-cell RNA-seq data processing" section. 

## Step 1.4 Data normalization and feature selection
After processing scRNA-seq and scDNA-seq data, we can obtain two data matrices for each patient. One matrix is the scRNA-seq cell-by-gene matrix with columns as cell ids and rows as gene ids, named `rna_raw`. The other matrix is CNV cell-by-gene matrix with columns as cell ids and rows as gene ids, named `cnv_raw`. For illustrating how we processed the data and how to run MaCroDNA, we adopted [NCI-N87 from CCNMF](https://github.com/XQBai/CCNMF).

In [3]:
import utils
import pandas as pd
import numpy as np
import os
import copy
import subprocess
# read data as dataframe
DataDir = "Data/"
rna_raw = pd.read_csv(DataDir+"rna_raw.csv",index_col=0)
dna_raw = pd.read_csv(DataDir+"cnv_raw.csv",index_col=0)

Considering different processing methods for Seurat and clonealign, we applied 4 different processing strategies, denoted as noX_genes_raw, all_genes_raw,all_genes_log, and 2000_genes_log.

### Step 1.4.1 noX_genes_raw
The first one is the clonealign’s preprocessing, tagged with noX genes raw. In this strategy, genes expressed in less than 1% of cells are removed and genes on chromosome X are also removed.

In [4]:
# remove genes on chrX
gene_file = pd.read_csv(DataDir+"hg38_gene_ordering.txt",sep="\t",
                        index_col=0, header=None,
                        names=["chr","start","end"])
chrX_gene = gene_file[gene_file["chr"]=="chrX"].index.to_list()
dna_noX_raw = dna_raw.drop(set(chrX_gene).intersection(dna_raw.index), 
                           axis=0, inplace=False)
dna_noX_raw = dna_noX_raw.round().astype(int)

# filter out genes expressed in less than 1% of cells,
# keep the genes appear in both RNA and DNA data
dna_noX_genes_raw,rna_noX_genes_raw = utils.find_genes(dna_noX_raw,rna_raw)

# As long as there is gene has 0 copy number in one cell,
# there is the error "Initial elbo is NA" when running clonealign
# (as mentioned in https://github.com/kieranrcampbell/clonealign/issues/4)
# To avoid this, two strategies are tired: add1 and rm0

# add1
# add pseudo 1 to all copy number
dna_noX_genes_raw_add1 = utils.pseudo_1(dna_noX_genes_raw)
rna_noX_genes_raw_add1 = rna_noX_genes_raw.loc[dna_noX_genes_raw_add1.index]
dna_noX_genes_raw_add1.to_csv(DataDir+"dna_noX_genes_raw_add1.csv")
rna_noX_genes_raw_add1.to_csv(DataDir+"rna_noX_genes_raw_add1.csv")

# rm0
# remove genes that have 0 copy number in at least one cell
dna_noX_genes_raw_rm0 = utils.rm_0_gene(dna_noX_genes_raw)
rna_noX_genes_raw_rm0 = rna_noX_genes_raw.loc[dna_noX_genes_raw_rm0.index]
dna_noX_genes_raw_rm0.to_csv(DataDir+"dna_noX_genes_raw_rm0.csv")
rna_noX_genes_raw_rm0.to_csv(DataDir+"rna_noX_genes_raw_rm0.csv")

# rm0 is chosen as the process method in the paper
dna_noX_genes_raw_rm0.to_csv(DataDir+"dna_noX_genes_raw.csv")
rna_noX_genes_raw_rm0.to_csv(DataDir+"rna_noX_genes_raw.csv")

### Step 1.4.2 all_genes_raw
In this strategy, only genes expressed in less than 1% of cells are removed. The genes on chromosome X are kept.

In [5]:
# filter out genes expressed in less than 1% of cells,
# keep the genes appear in both RNA and DNA data
dna_all_genes_raw,rna_all_genes_raw = utils.find_genes(dna_raw,rna_raw)
dna_all_genes_raw.to_csv(DataDir+"dna_all_genes_raw.csv")
rna_all_genes_raw.to_csv(DataDir+"rna_all_genes_raw.csv")

### Step 1.4.3 2000_genes_log
The third strategy is Seurat’s preprocessing method, named as 2000_genes_log. This includes log transformation of the data, z-score normalization with 10,000 factor size, feature selection, and the selection of the top 2,000 genes. All these steps were applied sequentially to the input.

In [8]:
subprocess.call("/usr/bin/Rscript --vanilla seurat_process.R 2000",
               shell=True,stdout=subprocess.DEVNULL)

The legacy packages maptools, rgdal, and rgeos, underpinning this package
will retire shortly. Please refer to R-spatial evolution reports on
https://r-spatial.org/r/2023/05/15/evolution4.html for details.
This package is now running under evolution status 0 
Attaching SeuratObject
Performing log-normalization
0%   10   20   30   40   50   60   70   80   90   100%
[----|----|----|----|----|----|----|----|----|----|
**************************************************|
Centering and scaling data matrix
  |======================================================================| 100%
Calculating gene variances
0%   10   20   30   40   50   60   70   80   90   100%
[----|----|----|----|----|----|----|----|----|----|
**************************************************|
Calculating feature variances of standardized and clipped values
0%   10   20   30   40   50   60   70   80   90   100%
[----|----|----|----|----|----|----|----|----|----|
**************************************************|
Perfo

0

### Step 1.4.1 all_genes_log
The last strategy includes the log transformation and z-score normalization (with 10,000451
factor size), but all genes were kept.

In [9]:
subprocess.call("/usr/bin/Rscript --vanilla seurat_process.R all",
               shell=True,stdout=subprocess.DEVNULL)

The legacy packages maptools, rgdal, and rgeos, underpinning this package
will retire shortly. Please refer to R-spatial evolution reports on
https://r-spatial.org/r/2023/05/15/evolution4.html for details.
This package is now running under evolution status 0 
Attaching SeuratObject
Performing log-normalization
0%   10   20   30   40   50   60   70   80   90   100%
[----|----|----|----|----|----|----|----|----|----|
**************************************************|
Centering and scaling data matrix
  |======================================================================| 100%
Calculating gene variances
0%   10   20   30   40   50   60   70   80   90   100%
[----|----|----|----|----|----|----|----|----|----|
**************************************************|
Calculating feature variances of standardized and clipped values
0%   10   20   30   40   50   60   70   80   90   100%
[----|----|----|----|----|----|----|----|----|----|
**************************************************|
Perfo

0

# Step 2. Run MaCroDNA
MaCroDNA (Matching Cross-Domain Nucleic Acids) is a correlation-based method to perform mapping between scRNA-seq gene expression and scDNA-seq copy number values. The following is the instruction of running MaCroDNA on DNA and RNA data. We used processed data from 2000_genes_log as an example.

In [10]:
from sklearn.cluster import AgglomerativeClustering
from macrodna import MaCroDNA
dna = pd.read_csv(DataDir+"dna_2000_genes_log.csv",index_col=0)
rna = pd.read_csv(DataDir+"rna_2000_genes_log.csv",index_col=0)

## Step 2.1 DNA cells clustering
MaCroDNA can map RNA cells to DNA cells, and it can also map RNA cells to DNA clones. To obtain clones from DNA sample, we used hierarchical clustering method as an example to get clones. The clustering methods mentioned in the paper can be found in clone_generation.ipynb

In [12]:
dna_clone = AgglomerativeClustering(n_clusters=2,metric="euclidean",
                                    linkage="ward").fit_predict(dna.to_numpy().T)
dna_clone = pd.DataFrame(list(zip(dna.columns,dna_clone)), columns=["cell","clone"])
dna_clone.to_csv("dna_clone.csv")

## Step 2.2 Run MaCroDNA on DNA clones

In [ ]:
model = MaCroDNA(rna_df=rna, dna_df=dna,dna_label=dna_clone)
rna_clone = model.cell2clone_assignment()
"""
rna_clone is dataframe. It has three columns. 
"cell" means the RNA cell id.
"predict_cell" is the DNA cell id. 
It means the DNA cell that is mapped to the corresponding RNA cell by MaCroDNA.
"predict_clone" is the clone id of the DNA cell in "predict_cell" column. 
It means the DNA clones that is assigned to the RNA cell by MaCroDNA.
"""

rna_clone.to_csv("macrodna_result.csv")

/home/xiru/Downloads/scripts/scripts/CRC_data_analysis/macrodna.py:87: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  self.dna_df = self.dna_df.loc[genes, :]
/home/xiru/Downloads/scripts/scripts/CRC_data_analysis/macrodna.py:88: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  self.rna_df = self.rna_df.loc[genes, :]


number of cells in dna data 724
number of cells in rna data 3246
number of genes in dna data 1998
number of genes in rna data 1998
4 350
MaCroDNA will be run for 5 steps
the smallest set has 724 number of cells
Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-30
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Core(TM) i9-10900K CPU @ 3.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 3971 rows, 2350104 columns and 7050312 nonzeros
Model fingerprint: 0x8be85cf0
Variable types: 0 continuous, 2350104 integer (2350104 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-05, 2e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Found heuristic solution: objective 89.7047505
Presolve removed 0 rows and 0 columns (presolve time = 5s) ...
Presolve removed 0 rows and 0 columns (